<a href="https://colab.research.google.com/github/jeHanCindy/-/blob/main/%ED%95%99%EC%8A%B5%EC%9A%A9_BigData_example3_LAB_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2.기출문제 (빅분기 실기 4회)

In [ ]:
# 데이터 다운로드

In [ ]:
!git clone https://github.com/Soyoung-Yoon/bigdata

#### 2-1. 신상 데이터
age 컬럼에 대해 아래의 과정을 수행하고 (다) 단계의 결과값을 출력하여라.
- (가) 제 1사분위수와 제 3사분위수를 구하기
- (나) 두 값의 차이 절댓값 구하기
- (다) 그 값의 소수점 버리기


In [ ]:
#'https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/basic1_data04.csv'  파일 사용

#### 2-2. Facebook 데이터

Facebook 데이터셋을 불러와 (loves반응 + wows반응)/(reactions반응) 비율이 0.4보다 크고 0.5보다 작으면서, type이 'video'인 데이터의 갯수를 구하여라
- 'https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/fb_data04.csv' 파일 사용

#### 2-3. 넷플릭스 데이터
netflix_subset 데이터셋에서 date_added가 2018년 1월이면서 country가 United Kingdom 단독 제작인 데이터의 개수를 구하여라
- 'https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/nf_data04.csv' 파일 사용

## 3.기출문제 (빅분기 실기 4회)
자동차 시장 세분화
- 자동차 회사는 새로운 전략을 수립하기 위해 4개의 시장으로 세분화했습니다.
- 기존 고객 분류 자료를 바탕으로 신규 고객이 어떤 분류에 속할지 예측해주세요!
- 예측할 값(y): "Segmentation" (1,2,3,4)
- 평가: Macro f1-score
- 제출형식

        ID,Segmentation
        458989,1
        458994,2
        459000,3
        459003,4

- 다음의 데이터 파일 준비를 실행해서 x_train.csv, y_train.csv, x_test.csv 파일을 준비합니다.


### 3-01.데이터 불러오기, 탐색

In [1]:
# 데이터 파일 준비
# 그냥 실행해서 사용 파일을 생성하도록 합니다.
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/train_04.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/test_04.csv')
train.head(2)
X_train = train.iloc[:, :-1]
Y_train = train[['ID', 'Segmentation']]
X_train.to_csv('X_train.csv', index=False)
Y_train.to_csv('Y_train.csv', index=False)
test.to_csv('X_test.csv', index=False)
print('파일을 준비했습니다!  X_train.csv, Y_train.csv, X_test.csv !')

파일을 준비했습니다!  X_train.csv, Y_train.csv, X_test.csv !


In [2]:
# [0] 사용 라이브러리 import
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import f1_score, roc_auc_score

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

# 데이타가 많은경우 모두 출력 안되고 ... 으로 생략해서 출력됨.
pd.set_option('display.max_rows',500)    #출력할 max row를 지정
pd.set_option('display.max_columns',20)  #출력할 max columns를 지정
#출력 format 지정 - 소수점아래 2자리까지
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
# [1] x_train.csv, y_train.csv, x_test.csv 읽어오기
X = pd.read_csv('X_train.csv')
Y = pd.read_csv('Y_train.csv')
X_submission = pd.read_csv('X_test.csv')

# print(X.shape, Y.shape, X_submission.shape)
#print(X.head(2), Y.head(2), X_submission.head(2), sep = '\n\n')

# [2] 데이터 전처리
# y값의 범주별 분포 확인 - 안 해도 문제는 없음
'''
결측치 없음
1   Gender           6665 non-null   object -> 0, 1
2   Ever_Married     6665 non-null   object -> 0, 1
4   Graduated        6665 non-null   object -> 0, 1
5   Profession       6665 non-null   object 9개 ['Healthcare' 'Engineer' 'Lawyer' 'Artist' 'Doctor' 'Homemaker' 'Entertainment' 'Marketing' 'Executive'] -> LE
7   Spending_Score   6665 non-null   object 3개 ['Low' 'High' 'Average'] -> OH
9   Var_1            6665 non-null   object 7개 ['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' 'Cat_5'] -> LE
'''
dfX = pd.concat([X, X_submission], axis = 0)
#dfX.info()
#dfX.isna().sum()

# print(dfX['Profession'].unique())
# print(dfX['Spending_Score'].unique())
#print(dfX[['Var_1']].value_counts())
#print(Y['Segmentation'].value_counts(normalize = True))

# 0, 1, LE
for x in ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Var_1'] :
  dfX[x] = dfX[x].astype('category').cat.codes

temp = pd.get_dummies(dfX['Spending_Score'])
# 만약 OH인코딩할게 많은 경우에는 아예 dfX[['컬럼명1', '컬럼명2']]로 입력
dfX = pd.concat([dfX, temp], axis = 1)
dfX = dfX.drop(columns = 'Spending_Score') # 새로운 컬럼이 생성되기 때문에 범주형 변수는 drop
dfX.info()

####################### 전처리 완료



<class 'pandas.core.frame.DataFrame'>
Int64Index: 8819 entries, 0 to 2153
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8819 non-null   int64  
 1   Gender           8819 non-null   int8   
 2   Ever_Married     8819 non-null   int8   
 3   Age              8819 non-null   int64  
 4   Graduated        8819 non-null   int8   
 5   Profession       8819 non-null   int8   
 6   Work_Experience  8819 non-null   float64
 7   Family_Size      8819 non-null   float64
 8   Var_1            8819 non-null   int8   
 9   Average          8819 non-null   uint8  
 10  High             8819 non-null   uint8  
 11  Low              8819 non-null   uint8  
dtypes: float64(2), int64(2), int8(5), uint8(3)
memory usage: 413.4 KB


In [4]:
# [3] 데이터 모델링 & 평가
# [3-1] 데이터 분리
X1 = dfX.iloc[:X.shape[0], :]
X_submission1 = dfX.iloc[X.shape[0]:, :]
Y1 = Y['Segmentation']

# print(X1.shape, X_submission1.shape, Y1.shape)
# (6665, 12) (2154, 12) (6665,)

# dfX.describe()
scaler = MinMaxScaler()
X1 = scaler.fit_transform(X1)
X_submission1 = scaler.transform(X_submission1) # 왜냐하면 fit까지 있는 경우에는 X_submission1끼지 값을 변경하기 때문

r = train_test_split(X1, Y1, test_size = 0.25, stratify = Y1, random_state = 1127)
x_train, x_test, y_train, y_test = r
# print([x.shape for x in r])
# [(4998, 12), (1667, 12), (4998,), (1667,)]

# [3-2] 데이터 모델링
model1 = LogisticRegression().fit(x_train, y_train) # 훈련용 데이터
y_pred = model1.predict(x_test) # X의 테스트 데이터로 예측

a = model1.score(x_train, y_train)
b = model1.score(x_test, y_test)
c = f1_score(y_test, y_pred, average = 'macro') # 큰 값이 좋은 경우
# 정확도, 훈련 데이터
# f1_score(y_true, y_pred, average='macro') <- 이렇게 쓰라고 help에 나옴
print(f'{a:.4f} {b:.4f} {c:.4f}')
# 0.5060 0.5063 0.4764

####################
model2 = KNeighborsClassifier(3).fit(x_train, y_train) # 훈련용 데이터
y_pred = model2.predict(x_test) # X의 테스트 데이터로 예측

a = model2.score(x_train, y_train)
b = model2.score(x_test, y_test)
c = f1_score(y_test, y_pred, average = 'macro') # 큰 값이 좋은 경우
# 정확도, 훈련 데이터
# f1_score(y_true, y_pred, average='macro') <- 이렇게 쓰라고 help에 나옴
print(f'{a:.4f} {b:.4f} {c:.4f}')
# 0.6829 0.4547 0.4518

####################
model3 = DecisionTreeClassifier(max_depth = 5).fit(x_train, y_train) # 훈련용 데이터
y_pred = model3.predict(x_test) # X의 테스트 데이터로 예측

a = model3.score(x_train, y_train)
b = model3.score(x_test, y_test)
c = f1_score(y_test, y_pred, average = 'macro') # 큰 값이 좋은 경우
# 정확도, 훈련 데이터
# f1_score(y_true, y_pred, average='macro') <- 이렇게 쓰라고 help에 나옴
print(f'{a:.4f} {b:.4f} {c:.4f}')
# 0.4988 0.5009 0.4977
# max_depth = 4 | 0.5246 0.5243 0.4991
# max_depth = 5 | 0.5358 0.5261 0.5210

####################
model4 = RandomForestClassifier(max_depth = 3).fit(x_train, y_train) # 훈련용 데이터
y_pred = model4.predict(x_test) # X의 테스트 데이터로 예측

a = model4.score(x_train, y_train)
b = model4.score(x_test, y_test)
c = f1_score(y_test, y_pred, average = 'macro') # 큰 값이 좋은 경우
# 정확도, 훈련 데이터
# f1_score(y_true, y_pred, average='macro') <- 이렇게 쓰라고 help에 나옴
print(f'{a:.4f} {b:.4f} {c:.4f}')
# 0.5194 0.5027 0.4703




0.5060 0.5063 0.4764
0.6829 0.4547 0.4518
0.5358 0.5261 0.5210
0.5084 0.4895 0.4425


In [5]:
# [4] 제출
predict = model4.predict(X_submission1) #!!!!!! 꼭 제출용 데이터로 변경해줘야 함
# predict : 예측값
submission = pd.DataFrame({'Segmentation':predict}) # 확률로 제출하라 등의 지시문이 있는 경우 그것대로 저장
submission.to_csv('수험번호.csv', index = False)

In [6]:
# 확인
temp = pd.read_csv('수험번호.csv')
temp.tail(5)

,Segmentation
2149,1
2150,4
2151,1
2152,3
2153,4
